## Part 14: export data for a web map
michael babb
2025 10 01

In [27]:
# standard
import os

In [28]:
# external
import geopandas as gpd
import numpy as np
import pandas as pd

In [29]:
# custom
import run_constants as rc
from geo_data_io.fc_df_spatial import  check_MultiLineStrings
from geo_data_io.fileio import write_json

# load the added street data

In [30]:
fpn = os.path.join(rc.OUTPUT_FILE_PATH, rc.S13_MISSING_OUT_FILE_NAME)

In [31]:
gdf = gpd.read_file(filename = fpn)

# dissolve and aggregate
This is necessary so that the geometry is grouped together

In [32]:
# dissolve - this also aggregates
col_names = ['ord_stname_type_group','snd_group', 'street_status_reclass', 'group_id', 'dist_miles', 'geometry']
diss_gdf = gdf[col_names].dissolve(by = col_names[:-2],
                     aggfunc =  ['sum'], as_index = False)

In [33]:
diss_gdf.head()

,ord_stname_type_group,snd_group,street_status_reclass,group_id,geometry,"(dist_miles, sum)"
0,100TH PL NW,0,0_0,1,"MULTILINESTRING ((-122.36206 47.7035, -122.363...",0.201049
1,100TH PL SW,1,0_0,2,"LINESTRING (-122.3403 47.51334, -122.34036 47....",0.035254
2,100TH ST NW_N_NE,0,2_0,1,"LINESTRING (-122.38387 47.70155, -122.3823 47....",0.072951
3,100TH ST NW_N_NE,1,2_0,1,"LINESTRING (-122.36478 47.70151, -122.36208 47...",0.125399
4,100TH ST NW_N_NE,2,1_0,1,"MULTILINESTRING ((-122.38757 47.70155, -122.38...",0.273002


In [34]:
# set column names
col_names = ['ord_stname_type_group','snd_group', 'street_status_reclass', 'group_id', 'geometry', 'dist_miles']
diss_gdf.columns = col_names

In [35]:
# add a column to count the number of records - this will be summed later
diss_gdf['n_segments'] = 1
# shorten column names. This reduces the file size a bit because of how json stores data
# it's better to repeat 'osntg' as opposed to 'ord_stname_type_group'
col_names = ['osntg','sndg', 'ss', 'gi', 'geometry', 'dm', 'ns']
diss_gdf.columns = col_names

In [36]:
# let's try dropping some columns
drop_col_names = ['sndg', 'gi']
diss_gdf = diss_gdf.drop(labels=drop_col_names, axis = 1)

In [37]:
# perform another dissolve in order to get the right count of segments
# this is necessary because we want continuous segments to be counted as one
diss_gdf.columns
col_names = ['osntg', 'ss', 'dm', 'ns']
diss_gdf = diss_gdf.dissolve(by = col_names[:2], aggfunc =  ['sum'], as_index = False)
diss_gdf.head()

,osntg,ss,geometry,"(dm, sum)","(ns, sum)"
0,100TH PL NW,0_0,"MULTILINESTRING ((-122.36206 47.7035, -122.363...",0.201049,1
1,100TH PL SW,0_0,"LINESTRING (-122.3403 47.51334, -122.34036 47....",0.035254,1
2,100TH ST NW_N_NE,1_0,"MULTILINESTRING ((-122.38757 47.70155, -122.38...",4.535595,9
3,100TH ST NW_N_NE,2_0,"MULTILINESTRING ((-122.38387 47.70155, -122.38...",0.653439,7
4,100TH ST NW_N_NE,3_1,"LINESTRING (-122.33512 47.70142, -122.32852 47...",0.307315,1


In [38]:
# rename 
diss_gdf.columns = ['osntg', 'ss', 'geometry', 'dm', 'ns']
diss_gdf.shape
diss_gdf.head()

,osntg,ss,geometry,dm,ns
0,100TH PL NW,0_0,"MULTILINESTRING ((-122.36206 47.7035, -122.363...",0.201049,1
1,100TH PL SW,0_0,"LINESTRING (-122.3403 47.51334, -122.34036 47....",0.035254,1
2,100TH ST NW_N_NE,1_0,"MULTILINESTRING ((-122.38757 47.70155, -122.38...",4.535595,9
3,100TH ST NW_N_NE,2_0,"MULTILINESTRING ((-122.38387 47.70155, -122.38...",0.653439,7
4,100TH ST NW_N_NE,3_1,"LINESTRING (-122.33512 47.70142, -122.32852 47...",0.307315,1


In [39]:
# format the osntg output
def format_osntg(sn):
    if '_' in sn:
        pos = sn.rfind(' ') 
        osn = sn[:pos]
        #ost = sn[pos + 1:].replace('_', '] [')
        #outcome = osn + ' : [' + ost + ']'
        ost = sn[pos + 1:].replace('_', ' | ')
        outcome = osn + ': ' + ost

    else:
        outcome = sn
    return outcome


In [40]:
diss_gdf['osntg'] = diss_gdf['osntg'].map(format_osntg)
diss_gdf.head()

,osntg,ss,geometry,dm,ns
0,100TH PL NW,0_0,"MULTILINESTRING ((-122.36206 47.7035, -122.363...",0.201049,1
1,100TH PL SW,0_0,"LINESTRING (-122.3403 47.51334, -122.34036 47....",0.035254,1
2,100TH ST: NW | N | NE,1_0,"MULTILINESTRING ((-122.38757 47.70155, -122.38...",4.535595,9
3,100TH ST: NW | N | NE,2_0,"MULTILINESTRING ((-122.38387 47.70155, -122.38...",0.653439,7
4,100TH ST: NW | N | NE,3_1,"LINESTRING (-122.33512 47.70142, -122.32852 47...",0.307315,1


In [41]:
# see if we can collapse MultiLineStrings to LineStrings
diss_gdf['geometry'].geom_type.value_counts()


MultiLineString    2005
LineString         1148
Name: count, dtype: int64

In [42]:
diss_gdf['geometry'] = diss_gdf['geometry'].map(check_MultiLineStrings)
diss_gdf['geometry'].geom_type.value_counts()

LineString         1681
MultiLineString    1472
Name: count, dtype: int64

In [43]:
diss_gdf.head()

,osntg,ss,geometry,dm,ns
0,100TH PL NW,0_0,"LINESTRING (-122.36206 47.7035, -122.36342 47....",0.201049,1
1,100TH PL SW,0_0,"LINESTRING (-122.3403 47.51334, -122.34036 47....",0.035254,1
2,100TH ST: NW | N | NE,1_0,"MULTILINESTRING ((-122.38757 47.70155, -122.38...",4.535595,9
3,100TH ST: NW | N | NE,2_0,"MULTILINESTRING ((-122.38387 47.70155, -122.38...",0.653439,7
4,100TH ST: NW | N | NE,3_1,"LINESTRING (-122.33512 47.70142, -122.32852 47...",0.307315,1


In [44]:
# recode the street segments
ss_recode_dict = {'0_0':0,
                  '1_0':1,
                  '2_0':2,
                  '3_0':3,
                  '2_1':4,
                  '3_1':5
}

diss_gdf['ss'] = diss_gdf['ss'].map(ss_recode_dict)

In [45]:
diss_gdf['ss'].value_counts()

ss
0    1117
1     699
2     636
4     380
3     304
5      17
Name: count, dtype: int64

In [46]:
diss_gdf.head()

,osntg,ss,geometry,dm,ns
0,100TH PL NW,0,"LINESTRING (-122.36206 47.7035, -122.36342 47....",0.201049,1
1,100TH PL SW,0,"LINESTRING (-122.3403 47.51334, -122.34036 47....",0.035254,1
2,100TH ST: NW | N | NE,1,"MULTILINESTRING ((-122.38757 47.70155, -122.38...",4.535595,9
3,100TH ST: NW | N | NE,2,"MULTILINESTRING ((-122.38387 47.70155, -122.38...",0.653439,7
4,100TH ST: NW | N | NE,5,"LINESTRING (-122.33512 47.70142, -122.32852 47...",0.307315,1


In [47]:
# finally, rename the CNTR streets to just C
diss_gdf['osntg'] = diss_gdf['osntg'].str.replace('CNTR', 'C')
out_data = diss_gdf.to_json(drop_id=True, to_wgs84=True)
output_file_name = rc.S14_STREETS_JSON_FILE_NAME
write_json(json_data=out_data, output_file_path ='../maps',
               output_file_name = output_file_name, var_name = 'all_streets_diss')


all_streets_diss
